# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 5**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 5. PyTerrier - Neural Re-Ranking

Dans ce TP vous allez :

 - reclasser des documents en utilisant des modèles neuronaux comme KNRM, Vanilla BERT, EPIC et monoT5.


# 0. Installation


In [1]:
!pip install -q --upgrade python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup

# Installation des plugins Pyterrier  

Nous installons les plugins PyTerrier [OpenNIR](https://opennir.net/) et [monoT5](https://github.com/terrierteam/pyterrier_t5). Vous pouvez ignorer sans risque les erreurs de version des paquets.

In [2]:
!pip install -q --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# 1. Preliminary steps

In [3]:
import os
import tensorflow as tf

**[PyTerrier](https://github.com/terrier-org/pyterrier) initialization**

In [4]:
import pyterrier as pt
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


from pyterrier.measures import * # allow for natural measure names
import onir_pt

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 2. Indexation

In [5]:
!rm -rf ./terrier_cord19/*

In [6]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 s<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M s<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.1%| 147k/269M s<03:29, 1.29MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.5%| 1.44M/269M s<44.84s, 5.97MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 2.8%| 7.43M/269M s<13.03s, 20.1MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazon

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

<ipython-input-6-47146416b637>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


21:04:00.302 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
21:05:15.546 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

# 3. Re-Rankers

Commençons à explorer quelques méthodes neuronales de re-classement ! Nous pouvons les construire à partir de zéro en utilisant `onir_pt.reranker`.

Le modèle de re-ranking d'OpenNIR est composé de :
 - `ranker` (par exemple, `drmm`, `knrm`, ou `pacrr`). Ceci définit l'architecture neuronale de classement.
 - `vocab` (par exemple, `wordvec_hash`, ou `bert`). Ceci définit comment le texte est encodé par le modèle. Cette approche rend facile l'échange de différentes représentations de texte.

L'exécution de cette ligne prendra quelques minutes car elle télécharge et prépare les vecteurs de mots.

In [7]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2024-04-20 21:05:39,397][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2024-04-20 21:05:46,217][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2024-04-20 21:05:46,225][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [6.61s] [682M] [134MB/s]
[2024-04-20 21:05:46,233][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [6.84s]
[2024-04-20 21:05:46,234][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2024-04-20 21:06:05,356][WordvecHashVocab][DEBUG] [finished] extracting vecs [19.12s]
[2024-04-20 21:06:05,356][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2024-04-20 21:08:25,285][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:20]
[2024-04-20 21:08:25,637][WordvecHashVocab][DEBUG] [starting] writing

Voyons comment ces modèles fonctionnent pour le classement !

In [8]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(cord19, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])


In [9]:
topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [18.7kB] [41.6MB/s]
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: s] [1.14MB] [3.61MB/s]


In [10]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

In [11]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-20 21:09:01,366][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:09:01,938][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-20 21:09:03,518][onir_pt][DEBUG] [finished] batches: [1.58s] [188it] [119.04it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM,0.042539,0.112699,0.423870,0.360000


Cela ne fonctionne pas très bien car le modèle n'est pas entraîné ; il utilise des poids aléatoires pour combiner les scores de la matrice de similarité.

# 4. Entraînement du re-ranker

Vous pouvez entraîner des modèles de re-classement dans PyTerrier en utilisant la méthode `fit`.

In [12]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

[2024-04-20 21:09:11,874][onir_pt][DEBUG] [starting] validation
[2024-04-20 21:09:11,875][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:09:11,876][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-20 21:09:12,057][onir_pt][DEBUG] [finished] batches: s] [63it] [348.38it/s]
[2024-04-20 21:09:12,185][onir_pt][DEBUG] [finished] validation s]
[2024-04-20 21:09:12,186][onir_pt][INFO] pre-validation: 0.0053
[2024-04-20 21:09:12,193][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:09:12,194][onir_pt][DEBUG] [starting] training
[2024-04-20 21:09:12,195][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-20 21:09:12,722][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:13,258][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:14,365][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:15,623][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:16,139][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:17,375][onir_pt][DEBUG] not enough negs
[2024-04-20 21:09:18,753][onir_pt][DEBUG] [finished] train pairs: [6.56s] [1024it] [156.16it/s]
[2024-04-20 21:09:18,758][onir_pt][DEBUG] [finished] training [6.56s]
[2024-04-20 21:09:18,759][onir_pt][INFO] training   it=0 loss=0.2451
[2024-04-20 21:09:18,759][onir_pt][DEBUG] [starting] validation
[2024-04-20 21:09:18,760][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:09:18,761][onir_pt][DEBUG] [starting] batches


...

[2024-04-20 21:11:47,769][onir_pt][DEBUG] [finished] batches: s] [63it] [211.96it/s]
[2024-04-20 21:11:48,032][onir_pt][DEBUG] [finished] validation s]
[2024-04-20 21:11:48,033][onir_pt][INFO] validation it=22 map=0.0060 ndcg=0.0154 P_10=0.0700
[2024-04-20 21:11:48,034][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:11:48,034][onir_pt][DEBUG] [starting] training
[2024-04-20 21:11:48,035][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-20 21:11:48,747][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:49,118][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:49,462][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:50,282][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:50,827][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:53,313][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:53,953][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:54,270][onir_pt][DEBUG] not enough negs
[2024-04-20 21:11:54,279][onir_pt][DEBUG] [finished] train pairs: [6.24s] [1024it] [164.02it/s]
[2024-04-20 21:11:54,282][onir_pt][DEBUG] [finished] training [6.25s]
[2024-04-20 21:11:54,283][onir_pt][INFO] training   it=23 loss=0.2235
[2024-04-20 21:11:54,283][onir_pt][DEBUG] [starting] validation
[2024-04-20 21:11:54,284][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:11:54,285][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-20 21:11:54,515][onir_pt][DEBUG] [finished] batches: s] [63it] [274.31it/s]
[2024-04-20 21:11:54,745][onir_pt][DEBUG] [finished] validation s]
[2024-04-20 21:11:54,746][onir_pt][INFO] validation it=23 map=0.0060 ndcg=0.0153 P_10=0.0700
[2024-04-20 21:11:54,747][onir_pt][INFO] early stopping; model reverting back to it=3


In [13]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-20 21:11:59,407][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:11:59,408][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-20 21:11:59,856][onir_pt][DEBUG] [finished] batches: s] [188it] [420.76it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM (trained),0.052371,0.121796,0.521478,0.453333


Le résultat est plus élévé, mais cela reste moins bon que le TFIDF. Proposez une hypothèse sur le problème.

le modèle KNRM entraîné n'ait pas donné de résultats aussi bons que le modèle TF-IDF seul, ce qui est contre-intuitif étant donné que l'objectif de l'entraînement du modèle est généralement d'améliorer les performances par rapport à une méthode de base comme TF-IDF. Voici quelques hypothèses possibles sur le problème :

1. **Surajustement (overfitting) du modèle KNRM** : le modèle KNRM ait surajusté les données d'entraînement, ce qui signifie qu'il a trop bien appris les exemples d'entraînement spécifiques mais qu'il ne généralise pas bien aux nouvelles données. Cela peut se produire si le modèle est trop complexe par rapport à la quantité de données d'entraînement disponibles.

2. **Paramètres d'entraînement non optimaux** : Les performances du modèle KNRM peuvent être sensibles aux paramètres d'entraînement tels que le taux d'apprentissage, le nombre d'époques d'entraînement, la taille du lot, etc. Il est possible que les paramètres que vous avez utilisés ne soient pas optimaux pour votre ensemble de données spécifique.

3. **Mauvaise initialisation des poids** : L'initialisation des poids du modèle KNRM peut jouer un rôle important dans ses performances. Si les poids sont initialisés de manière inappropriée, cela peut conduire à des difficultés lors de la convergence de l'entraînement ou à des performances médiocres sur les données de test.

4. **Déséquilibre des données d'entraînement** : Si les données d'entraînement sont déséquilibrées, c'est-à-dire si elles contiennent un nombre disproportionné de exemples positifs ou négatifs, cela peut affecter les performances du modèle KNRM, en particulier s'il n'est pas correctement traité.

5. **Erreur dans le prétraitement des données** : Il est également possible qu'il y ait eu une erreur dans le prétraitement des données avant l'entraînement du modèle KNRM, ce qui pourrait avoir un impact sur ses performances.



# 5. Vanilla BERT

Les modèles linguistiques contextualisés, tels que [BERT](https://arxiv.org/abs/1810.04805), sont des modèles neuronaux beaucoup plus puissants qui se sont avérés efficaces pour le classement.

Nous allons essayer d'utiliser une version "vanille" (ou "mono") du modèle BERT. Le modèle BERT est pré-entraîné pour la modélisation du langage et la prédiction de la phrase suivante.

In [14]:
#del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

100%|██████████| 231508/231508 s<0ms, 1291708.75B/s]
100%|██████████| 433/433 s<0ms, 301133.08B/s]
100%|██████████| 440473133/440473133 [10.20s<0ms, 43203888.06B/s] 


Voyons comment ce modèle se comporte sur la collection TREC COVID.

In [15]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-20 21:12:49,649][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:12:49,807][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-20 21:13:11,183][onir_pt][DEBUG] [finished] batches: [21.37s] [188it] [ 8.80it/s]


,name,nDCG,nDCG@10,AP(rel=2),P(rel=2)@10,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,AP(rel=2) +,AP(rel=2) -,AP(rel=2) p-value,P(rel=2)@10 +,P(rel=2)@10 -,P(rel=2)@10 p-value
0,TFIDF,0.123589,0.595818,0.054568,0.546667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TFIDF >> VBERT,0.118277,0.479929,0.049198,0.420000,3.0,12.0,0.11542,4.0,11.0,0.02287,3.0,12.0,0.012239,3.0,12.0,0.024362


In [16]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

[2024-04-20 21:13:40,473][onir_pt][DEBUG] [starting] validation
[2024-04-20 21:13:40,474][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:13:40,478][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-20 21:13:47,636][onir_pt][DEBUG] [finished] batches: [7.16s] [63it] [ 8.80it/s]
[2024-04-20 21:13:47,759][onir_pt][DEBUG] [finished] validation [7.29s]
[2024-04-20 21:13:47,760][onir_pt][INFO] pre-validation: 0.0059
[2024-04-20 21:13:49,356][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:13:49,360][onir_pt][DEBUG] [starting] training
[2024-04-20 21:13:49,361][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-20 21:13:53,621][onir_pt][DEBUG] not enough negs
[2024-04-20 21:14:12,199][onir_pt][DEBUG] not enough negs
[2024-04-20 21:14:50,112][onir_pt][DEBUG] not enough negs
[2024-04-20 21:15:30,627][onir_pt][DEBUG] not enough negs
[2024-04-20 21:15:47,380][onir_pt][DEBUG] not enough negs
[2024-04-20 21:16:25,391][onir_pt][DEBUG] not enough negs
[2024-04-20 21:17:05,278][onir_pt][DEBUG] [finished] train pairs: [03:16] [1024it] [ 5.23it/s]
[2024-04-20 21:17:05,280][onir_pt][DEBUG] [finished] training [03:16]
[2024-04-20 21:17:05,284][onir_pt][INFO] training   it=0 loss=0.2504
[2024-04-20 21:17:05,287][onir_pt][DEBUG] [starting] validation
[2024-04-20 21:17:05,287][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:17:05,295][onir_pt][DEBUG] [starting] batches


...

batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-20 21:31:01,664][onir_pt][DEBUG] [finished] batches: [7.23s] [63it] [ 8.72it/s]
[2024-04-20 21:31:01,791][onir_pt][DEBUG] [finished] validation [7.37s]
[2024-04-20 21:31:01,793][onir_pt][INFO] validation it=4 map=0.0059 ndcg=0.0156 P_10=0.0700
[2024-04-20 21:31:01,793][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:31:01,798][onir_pt][DEBUG] [starting] training
[2024-04-20 21:31:01,799][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-20 21:31:04,933][onir_pt][DEBUG] not enough negs


KeyboardInterrupt: 

In [17]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-20 21:31:13,735][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:31:13,741][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-20 21:31:35,311][onir_pt][DEBUG] [finished] batches: [21.57s] [188it] [ 8.72it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> VBERT,0.046396,0.116309,0.497435,0.413333


Comme nous le constatons, bien que le modèle soit pré-entraîné, il n'obtient pas de très bons résultats. Cela est dû au fait qu'il n'est pas réglé pour la tâche de classement par pertinence.

Cependant, nous pouvons entraîner le modèle pour le classement (comme indiqué ci-dessus pour KNRM).

# 6. monoT5

Le modèle [monoT5](https://arxiv.org/abs/2003.06713) évalue les documents à l'aide d'un modèle de langage causal. Voyons comment cette approche fonctionne sur TREC COVID.

La classe `MonoT5ReRanker` de `pyterrier_t5` charge automatiquement une version du classeur monoT5 qui est entraînée sur le jeu de données MS MARCO passage.

In [18]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k s<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M s<?, ?B/s]

config.json:   0%|          | 0.00/1.21k s<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.84k s<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M s<?, ?B/s]

In [19]:
pipeline = (tfidf >> get_text >> monoT5)
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

monoT5:   0%|          | 0/188 s<?, ?batches/s]

,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10,mrt
0,TFIDF,0.054568,0.123589,0.595818,0.546667,79.017215
1,TFIDF >> T5,0.058684,0.128119,0.685513,0.600000,1814.514595


Comme on pouvait s'y attendre, les résultats sont bien meilleurs en termes de NDCG@10 (0.5958 vs 0.6855).

# Tâche pratique

Comme pour le TP precedent, utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Vous pouvez le télécharger le dataset en utilisant les lines de code ci-dessous.

In [20]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train')
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/17918ed23cd04fb15047f73e6c3bd9d9
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip
[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: [1.91s] [17.9MB] [9.40MB/s]
[INFO] [finished] opening zip file [2.75s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


In [21]:
# réponse

import os
from itertools import chain

pt_index_path = './terrier_fiqa'

if not os.path.exists(pt_index_path + "/data.properties"):
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    index_ref = indexer.index(chain(fiqa['train'].get_corpus_iter(),
                                    fiqa['valid'].get_corpus_iter(),
                                    fiqa['test'].get_corpus_iter()),
                              fields=('text',),
                              meta=('docno',))

else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
docs_iter: 100%|████████████████████████| 57638/57638 [1.69s<0ms, 34056.67doc/s]
[INFO] [finished] docs_iter: [1.69s] [57638doc] [34034.81doc/s]
[INFO] [finished] building docstore [1.70s]


beir/fiqa/train documents:   0%|          | 0/57638 s<?, ?it/s]

beir/fiqa/dev documents:   0%|          | 0/57638 s<?, ?it/s]

beir/fiqa/test documents:   0%|          | 0/57638 s<?, ?it/s]

<ipython-input-21-c2bdfa48e16a>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(chain(fiqa['train'].get_corpus_iter(),


21:32:36.791 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (7915) - further warnings are suppressed
21:33:24.418 [ForkJoinPool-2-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 100 is not unique: 115281,57643
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java

In [22]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract',config={'max_train_it': 10,'random_seed': 42})


[2024-04-20 21:33:36,189][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2024-04-20 21:33:46,792][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [10.60s]


In [ ]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,

    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-20 21:34:45,856][onir_pt][DEBUG] using GPU (deterministic)
[2024-04-20 21:34:45,872][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/8100 s<?, ?it/s]

In [ ]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

In [ ]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

** Question**. Identifiez-vous des avantages par rapport à l'utilisation de Learning to Rank ?

---



**réponse**: Amelioration des resultats de recherche et des reponses parce que la base de donnee est une base de donnee de question reponse